In [2]:
import os
import torch
import torchsummary
from models import CNN_2d
from train_2d import train_datasets

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
cnn_2d = CNN_2d()
cnn_2d = cnn_2d.to(device)
torchsummary.summary(cnn_2d, (1, 512, 512))
del cnn_2d

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 512, 512]             160
         AvgPool2d-2         [-1, 16, 256, 256]               0
            Conv2d-3         [-1, 32, 256, 256]           4,640
         AvgPool2d-4         [-1, 32, 128, 128]               0
            Conv2d-5         [-1, 64, 128, 128]          18,496
         AvgPool2d-6           [-1, 64, 64, 64]               0
            Conv2d-7          [-1, 128, 64, 64]          73,856
         AvgPool2d-8          [-1, 128, 32, 32]               0
            Conv2d-9          [-1, 128, 32, 32]         147,584
        AvgPool2d-10          [-1, 128, 16, 16]               0
           Linear-11                 [-1, 2048]      67,110,912
           Linear-12                 [-1, 2048]       4,196,352
           Linear-13                    [-1, 9]          18,441
Total params: 71,570,441
Trainable para

In [4]:
# Params
batch_size = 20
epochs = 10
dataset_root = './datasets'
train_dataset = 'nodemcu-random-train2'
test_dataset = 'nodemcu-random-test2'

In [5]:
# train 
train_datasets(train_dataset, False, epochs)

Failed to get validation set


FileNotFoundError: [Errno 2] No such file or directory: './datasets/n'

In [ ]:
# Validate